In [1]:
import os
import imagej
from pathlib import Path
import sys
import csv
import glob

In [ ]:
"""Selects a file"""

%gui qt

from PyQt5.QtWidgets import QFileDialog

def get_filename(dir=None):
    """Select a file via a dialog and return the file name."""
    if dir is None: dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Select data file...", 
                dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]

In [ ]:
"""Selects a folder"""

%gui qt

from PyQt5.QtWidgets import QFileDialog

def get_folder(dir=None):
    """Select a folder via a dialog and return the folder path."""
    if dir is None: dir ='./'
    fname = QFileDialog.getExistingDirectory(None, "Select Directory...")
    return fname

In [ ]:
"""Modifies the macro file with the chosen directories"""

def modify_macro(macro, directory_in, directory_out):
    
    #To separate the method by OS
    #if os.name == 'nt':
    
    path_is_in=os.path.realpath(directory_in)
    path_is_in=path_is_in.replace('\\', '/')
    #print(path_is_in)
    path_is_out=os.path.realpath(directory_out)
    path_is_out=path_is_out.replace('\\', '/')
    #print(path_is_out)
    if not os.path.exists(macro):
        print('Macro file not there...')
        return False
    else:
        with open(macro, 'r+') as fp:
            lines=fp.readlines()
            for i, line in enumerate(lines):
                if line.startswith('inputDirectory'):
                    lines[i]='inputDirectory = "' + path_is_in + '/";\n'
                elif line.startswith('outputDirectory'):
                    lines[i]='outputDirectory = "' + path_is_out + '/";\n'
        with open(macro, 'r+') as fp:
            fp.writelines(lines)
        return True

In [ ]:
"""Makes a csv file with paths to images to analyze with Cellprofiler"""

def make_csv(var1):
    path_image=os.path.join(var1 + '/image_list.csv')
    print('Image list csv file:\n' + path_image)
    if not os.path.exists(path_image):
        with open(path_image, 'w') as fp:
            pass
    else:
        os.remove(path_image)
        with open(path_image,'w') as fp:
            pass
    path_image_files=os.path.join(var1, '*.tif')
    for file in glob.glob(path_image_files):
        #print('Writing...\n' + file)
        path_is=os.path.realpath(file)
        #print('Real path:\n' + path_is)
        with open(path_image, 'a') as image_list:
            dummywriter=csv.writer(image_list, delimiter=' ',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
            dummywriter.writerow([path_is])
    return path_image

In [2]:
"""Initialize ImageJ"""
"""Path to your ImageJ application"""

ij=imagej.init('C:/Users/presi/Documents/Fiji')
#ij=imagej.init('net.imagej:imagej+net.imagej:imagej-legacy')
#ij=imagej.init('sc.fiji:fiji')
ij.getVersion()

'2.0.0-rc-69/1.52p'

In [ ]:
"""Choose a macro file"""

macro_path=get_filename()
macro=open(macro_path)
macro=macro.read()
print('Macro selected:\n' + macro_path)

In [ ]:
"""Choose a folder with images to analyze with Fiji"""

fiji_in=get_folder()
print('Image is in:\n' + fiji_in)

In [ ]:
"""Create a folder where the images will be stored after Fiji"""

fiji_out = str(fiji_in + '_post_fiji') 
if not os.path.exists(fiji_out):
    os.makedirs(fiji_out)
print('Image will be saved in:\n' + fiji_out)

In [ ]:
"""Modify the macro file with new directories"""

try:
    modify_macro(macro_path, fiji_in, fiji_out)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

In [ ]:
"""Run the macro"""

ij.py.run_macro(macro)

In [ ]:
"""Create an output folder to save results after Cellprofiler"""
cp_out = str(fiji_in + '_post_Cellprofiler') 
if not os.path.exists(cp_out):
    os.makedirs(cp_out)
print('The final results will be saved in:\n' + cp_out)

In [ ]:
"""Create a csv file with image paths for Cellprofiler"""

path_image = make_csv(fiji_out)

In [ ]:
"""Choose a pipeline file"""

pipe = get_filename()
print('Pipeline:\n' + pipe)

In [ ]:
"""Modify the paths for shell command"""

pipe = pipe.replace(r'/','\\\\')
cp_in = path_image.replace(r'/','\\\\')
cp_out = cp_out.replace(r'/','\\\\')
print(pipe)
print(cp_in)
print(cp_out)

In [ ]:
"""Run Cellprofiler in shell"""
cmd = str('CellProfiler.lnk -c -r -p ' + pipe + ' --file-list=' + cp_in + ' -o ' + cp_out)
#print(cmd)
os.system(cmd)
    #os.system('cellprofiler -r -c -p ~/project/speckle_v3.cppipe --file-list=/home/h3/project/image_out_fiji/image_list.csv -o ~/project/image_out_cp')